In [104]:
import pandas as pd
import numpy as np
import random

In [105]:
#df = pd.read_csv (r'C:\Notebook scripts\Features_Variant_1.csv')
df = pd.read_csv ('Features_Variant_1.csv')

data_train=np.asarray(df.iloc[:,:])
#print(np.shape(data_train))

#shuffle before folding
np.random.shuffle(data_train)

In [106]:
#fold_1, fold_2, fold_3, fold_4, fold_5= np.array_split(data_train,5)
data_train_arr=np.array_split(data_train,5) #{[],[],[],[],[]}

y_train_arr=[]
x_train_arr=[]
for i in range(len(data_train_arr)):
    y_train_arr.append(data_train_arr[i][:,-1].reshape((-1, 1)) )
    x_train_arr.append(data_train_arr[i][:,:-1])


In [107]:
#потом нормализовать по ф-ле
mean_item=[]
std_item=[]
#print(x_train_arr[0][33,5])
for item in range(len(x_train_arr)):
    
    mean_item=np.mean(x_train_arr[item],axis=0)#(1,53)
    std_item=np.std(x_train_arr[item],axis=0)
    np.place(std_item, std_item==0, 1)#if std==0 then change it to 1 for division #std_item[std_item==0]=1 
    x_train_arr[item]=(x_train_arr[item]-mean_item)/std_item

In [108]:
def normalization(X):
    mean=np.mean(X,axis=0)#(1,53)
    std=np.std(X,axis=0)
    np.place(std, std==0, 1)#if std==0 then change it to 1 for division #std_item[std_item==0]=1 
    x_normalized=(X-mean)/std
    return x_normalized,mean,std
def normalization_with_params(X,mean,std):
    return (X-mean)/std

In [109]:
def linear_regression_predict(X,omega,omega0):
    prod=X.dot(omega)#(8190,1)
    #print(np.shape(y+prod))
    return prod+omega0
def root_mean_square_error(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    summands=np.power(y_actual-y_predict,2)
    #print(np.shape(summands))
    mse=np.sum(summands)/len(y_predict)
    return np.sqrt(mse)
   
def r_squared(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    nom=np.sum(np.power(y_actual-y_predict,2))
    denom=np.sum(np.power(y_actual-np.mean(y_actual,axis=0),2))
    return 1-nom/denom

In [110]:
def batch_gradient_descent(X,y_train,learning_rate,num_iterations):
                                   # dim omega
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_iterations): 
        N=len(X)
        error=y_train-linear_regression_predict(X,omega,omega0)
        omega=omega+learning_rate*2*((X.T).dot(error))/N
        omega0=omega0+learning_rate*2*np.sum(error)/N
        cost[it] = r_squared(X,omega,omega0,y_train)
    return omega,omega0,cost

In [111]:
# function to create a list containing mini-batches 
def create_mini_batches(X, y, batch_size): 
    mini_batches = [] 
    data = np.hstack((X, y)) 
    np.random.shuffle(data) 
    n_minibatches = data.shape[0] // batch_size 
    i = 0
  
    for i in range(n_minibatches + 1): 
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches.append((X_mini, Y_mini)) 
    if data.shape[0] % batch_size != 0: 
        mini_batch = data[i * batch_size:data.shape[0]] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches.append((X_mini, Y_mini)) 
    return mini_batches 

def mini_batch_gradient_descent(X,y_train,learning_rate,num_epoches,batch_size):
                                   # dim omega
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_epoches): 
        mini_batches = create_mini_batches(X, y_train, batch_size) 
        for mini_batch in mini_batches: 
            X_mini, y_mini = mini_batch 
            N=len(X_mini)
            error=y_mini-linear_regression_predict(X_mini,omega,omega0)
            omega=omega+learning_rate*2*((X_mini.T).dot(error))/N
            omega0=omega0+learning_rate*2*np.sum(error)/N
            cost.append(r_squared(X_mini,omega,omega0, y_mini)) 
    return omega,omega0,cost

In [112]:
r2=[]
rmse=[]
omega=[]
omega0=[]
for item in range(len(x_train_arr)):
    x_test=x_train_arr[item]
    y_test=y_train_arr[item]
    x_train_folds_arr=np.delete(x_train_arr,item)
    y_train_folds_arr=np.delete(y_train_arr,item)
    
    x_train=np.empty((0,x_train_arr[item].shape[1]), int)
    y_train=np.empty((0,y_train_arr[item].shape[1]), int)
    #print(np.shape(y_train_folds_arr))
    for i in range (len(x_train_folds_arr)):
        #print(np.shape(y_train_folds_arr[i]))
        #x_train_all55 =(np.vstack((x_train_all55,x_train_folds_arr[i])))
        x_train=np.concatenate((x_train,x_train_arr[i]))
        y_train=np.concatenate((y_train,y_train_arr[i]))
    #print(np.shape(y_train))
    #x_train,mean,std=normalization(x_train_rd)
    #x_test=normalization_with_params(x_test_rd,mean,std)
    #print(x)
    omega_item,omega0_item,cost=mini_batch_gradient_descent(x_train,y_train,learning_rate = 0.001,num_epoches=10,batch_size=32)
    r2.insert(item,r_squared(x_test,omega_item,omega0_item,y_test))
    rmse.insert(item,root_mean_square_error(x_test,omega_item,omega0_item,y_test))
    omega.insert(item,omega_item)
    omega0.insert(item,omega0_item)
    print(r2[item],rmse[item])

0.3496565066531112 32.334006523403616
0.3057181285256454 28.112272492595856
0.38882201361859825 25.196750726717518
0.31184029464034824 28.624073210886195
0.2391810401914547 31.734015011150262


In [113]:

# initialise data of lists. 
data = {'1':[rmse[0],r2[0],omega0[0]], 
        '2':[rmse[1],r2[1],omega0[1]],
        '3':[rmse[2],r2[2],omega0[2]],
        '4':[rmse[3],r2[3],omega0[3]],
        '5':[rmse[4],r2[4],omega0[4]],
        'E':[np.mean(rmse),np.mean(r2), np.mean(omega0)],
        'SD':[np.std(rmse),np.std(r2), np.std(omega0)]} 
 
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['RMSE', 'R^2', 'omega0']) 
df2 = pd.DataFrame(np.concatenate((np.hstack(omega),np.mean(omega,axis=0),np.std(omega,axis=0)),axis=1),
                   columns=['1', '2', '3','4','5','E','SD'])
df_col_merged =pd.concat([df, df2], axis=0)
df_col_merged

,1,2,3,4,5,E,SD
RMSE,32.334007,28.112272,25.196751,28.624073,31.734015,29.200224,2.599387
R^2,0.349657,0.305718,0.388822,0.311840,0.239181,0.319044,0.049827
omega0,7.380918,7.337582,7.189360,7.256847,7.390902,7.311122,0.077082
0,-0.371373,-0.507568,-0.893561,-1.024101,-1.022669,-0.763854,0.272493
1,-0.541648,-0.607559,-0.412262,-0.586842,-0.457608,-0.521184,0.074962
2,-1.627477,-1.395742,-2.415785,-2.164818,-2.506228,-2.022010,0.437673
3,-0.267416,-0.172701,-0.175227,-0.247202,-0.179581,-0.208425,0.040482
4,-0.317562,-0.426931,-0.319418,-0.817975,0.170800,-0.342217,0.315543
5,0.826271,0.939026,1.240229,0.979549,1.045738,1.006163,0.137062
6,0.835863,-0.045624,0.260141,-0.042063,0.134297,0.228523,0.324689
